In [ ]:
pip install -r requirements.txt

In [ ]:
# use the dotenv package to load the environment variables
from dotenv import load_dotenv
import os

# load the environment variables from the .env file
load_dotenv()

# get the environment variables and store them in variables
api_key = os.getenv('API_KEY')
base_url = os.getenv('BASE_URL')


# define the start and end date for the historical data
start_date = '2020-01-01'
end_date = '2020-12-31'

In [40]:
import requests
from datetime import date , timedelta

def get_apod_data(api_key, date):
    url = f"{base_url}/planetary/apod?api_key={api_key}&date={date}"
    try:
        response = requests.get(url)
        # check if the response is successful and raise an exception incase of an error
        response.raise_for_status()
        # parse the JSON response
        data = response.json()
        return {
            "date": data.get("date"),
            "title": data.get("title"),
            "url": data.get("url"),
            "explanation": data.get("explanation"),
            "media_type": data.get("media_type")
        }
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data for {date}: {e}")
    except KeyError:
        print("Unexpected response format")

# Get today's date  and fetch the data for today from the API to test the function
today = date.today()
today_data = get_apod_data(api_key, today)

print(today_data)

{'date': '2024-11-12', 'title': 'NGC 6888: The Crescent Nebula', 'url': 'https://apod.nasa.gov/apod/image/2411/Ngc6888Hoo_Aro_960.jpg', 'explanation': "How was the Crescent Nebula created?  Looking like an emerging space cocoon, the Crescent Nebula, visible in the center of the featured image, was created by the brightest star in its center.  A leading progenitor hypothesis has the Crescent Nebula beginning to form about 250,000 years ago.  At that time, the massive central star had evolved to become a Wolf-Rayet star (WR 136), shedding its outer envelope in a strong stellar wind, ejecting the equivalent of our Sun's mass every 10,000 years.  This wind impacted surrounding gas left over from a previous phase, compacting it into a series of complex shells, and lighting it up.  The Crescent Nebula, also known as NGC 6888, lies about 4,700 light-years away in the constellation of Cygnus.  Star WR 136 will probably undergo a supernova explosion sometime in the next million years.   Jigsaw 

In [41]:
# validate the response data by checking if the response is a dictionary and contains the expected keys and values 
import unittest

class TestGetApodData(unittest.TestCase):
    def test_get_apod_data(self):
        data = get_apod_data(api_key, date.today())
        self.assertIsInstance(data, dict)
        self.assertIn("date", data)
        self.assertIn("title", data)
        self.assertIn("url", data)
        self.assertIn("explanation", data)
        self.assertIn("media_type", data)
    
# Run the test
unittest.main(argv=[''], exit=False)


.
----------------------------------------------------------------------
Ran 1 test in 0.790s

OK


In [ ]:
from datetime import datetime
import time

# Function to fetch APOD data for multiple dates within a range
def fetch_multiple_apod_data(api_key, start_date, end_date):
    current_date = datetime.strptime(start_date, "%Y-%m-%d")
    end_date = datetime.strptime(end_date, "%Y-%m-%d")
    
    apod_data = []

    while current_date <= end_date:
        date_str = current_date.strftime("%Y-%m-%d")
        # Fetch data for the current date
        data = get_apod_data(api_key, date_str)
        if data:
            print(f"Fetched data for {date_str}")
            apod_data.append(data)
        # Move to the next date
        current_date += timedelta(days=1)
        # Delay to respect API rate limits
        time.sleep(1)  

    return apod_data

apod_data = fetch_multiple_apod_data(api_key, start_date, end_date)

In [42]:
# Better approach the fetch the data for multiple dates using start_date and end_date as query parameters in the API URL

def get_range_apod_data(api_key, start_date , end_date):
    url = f"{base_url}/planetary/apod?api_key={api_key}&start_date={start_date}&end_date={end_date}"
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        formatted_data = []
        for item in data:
            formatted_data.append({
                "date": item.get("date"),
                "title": item.get("title"),
                "url": item.get("url"),
                "explanation": item.get("explanation"),
                "media_type": item.get("media_type")
            })
        return formatted_data
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data for the range {start_date} to {end_date}: {e}")
    except KeyError:
        print("Unexpected response format")

In [53]:
import os
import json

def save_to_json(data, filename='apod_data.json'):
    try:
        # Verify if the data is a JSON-serializable list
        if not isinstance(data, list):
            raise ValueError("Data should be a list")

        # Check if the file exists
        if not os.path.exists(filename):
            # If the file does not exist, create it and add the data
            with open(filename, 'w') as file:
                json.dump(data, file, indent=4)
                file.write("\n")
            return
        
        # If the file exists, check if it contains any data
        with open(filename, 'r+') as file:
            if os.stat(filename).st_size == 0:
                # If the file is empty, add the data directly
                json.dump(data, file, indent=4)
                file.write("\n")
            else:
                # If the file has data, load existing data, concatenate with new data
                file.seek(0)
                existing_data = json.load(file)
                
                # Ensure existing data is a list
                if not isinstance(existing_data, list):
                    raise ValueError("Existing file data is not in list format")

                # Concatenate existing data with the new data
                updated_data = existing_data + data
                
                # Write the concatenated data back to the file
                file.seek(0)
                file.truncate(0)
                json.dump(updated_data, file, indent=4)
                file.write("\n")
                
    except ValueError as ve:
        print(f"Data validation error: {ve}")
    except IOError as e:
        print(f"Error writing to file {filename}: {e}")
    except json.JSONDecodeError:
        print(f"Error decoding JSON from file {filename}. Ensure the file format is correct.")


In [55]:
def fetch_and_save_apod_data(api_key, start_date, end_date):
    # using the get_range_apod_data function for better performance and to avoid rate limits
    range_data = get_range_apod_data(api_key, start_date, end_date)
    # to test the loop function, uncomment the line below and comment the line above
    # range_data = fetch_multiple_apod_data(api_key, start_date, end_date)
    if range_data:
        print(f"Fetched data for {len(range_data)} dates")
        # Save the data to a JSON file with the default filename
        save_to_json(range_data) 


fetch_and_save_apod_data(api_key, start_date, end_date)

Fetched data for 15 dates
